In [ ]:
import hddm

from patsy import dmatrix  # for generation of (regression) design matrices
import numpy as np         # for basic matrix operations
from pandas import Series  # to manipulate data-frames generated by hddm

: 

In [2]:
import sys
sys.stdout = open('ModelRecoveryOutput.txt', 'w')

In [15]:
n_subjects = 10
trials_per_level = 5 # and per stimulus

level1a = {'v':.3, 'a':2, 't':.3, 'sv':0, 'z':.5, 'sz':0, 'st':0}
level2a = {'v':.4, 'a':2, 't':.3, 'sv':0, 'z':.6, 'sz':0, 'st':0}
level3a = {'v':.5, 'a':2, 't':.3, 'sv':0, 'z':.7, 'sz':0, 'st':0}

In [16]:
data_a, params_a = hddm.generate.gen_rand_data({'level1': level1a,
                                                'level2': level2a,
                                                'level3': level3a},
                                                size=trials_per_level,
                                                subjs=n_subjects)

In [17]:
level1b = {'v':.3, 'a':2, 't':.3,'sv': 0, 'z':.5, 'sz': 0, 'st': 0}
level2b = {'v':.4, 'a':2, 't':.3,'sv': 0, 'z':.4, 'sz': 0, 'st': 0}
level3b = {'v':.5, 'a':2, 't':.3,'sv': 0, 'z':.3, 'sz': 0, 'st': 0}

In [18]:
data_b, params_b = hddm.generate.gen_rand_data({'level1': level1b,
                                                'level2': level2b,
                                                'level3': level3b},
                                                size=trials_per_level,
                                                subjs=n_subjects)

In [19]:
data_a['stimulus'] = Series(np.ones((len(data_a))), index=data_a.index)
data_b['stimulus'] = Series(np.ones((len(data_b)))*2, index=data_a.index)

In [20]:
mydata = data_a.append(data_b, ignore_index=True)

/var/folders/hl/dt523djx7_q7xjrthzjpdvc40000gn/T/ipykernel_53926/2454097547.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mydata = data_a.append(data_b, ignore_index=True)


In [21]:
mydata.head(n=50)

,rt,response,subj_idx,condition,stimulus
0,1.018445,0.0,0,level1,1.0
1,1.556326,1.0,0,level1,1.0
2,0.903633,1.0,0,level1,1.0
3,0.658032,1.0,0,level1,1.0
4,0.866491,1.0,0,level1,1.0
5,0.838610,1.0,0,level2,1.0
6,1.002273,1.0,0,level2,1.0
7,1.681373,1.0,0,level2,1.0
8,1.549412,0.0,0,level2,1.0
9,0.945523,0.0,0,level2,1.0


In [22]:
def z_link_func(x, data=mydata):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[x.index]}))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = stim - x
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

In [23]:
def z_link_func(x, data=mydata):
    stim = (np.asarray(dmatrix('0 + C(s, [[0], [1]])',
                              {'s': data.stimulus.loc[x.index]},return_type='dataframe'))
    )
    # Apply z = (1 - x) to flip them along 0.5
    z_flip = np.subtract(stim, x.to_frame())
    # The above inverts those values we do not want to flip,
    # so invert them back
    z_flip[stim == 0] *= -1
    return z_flip

In [24]:
z_reg = {'model': 'z ~ 1 + C(condition)', 'link_func': z_link_func}

In [25]:
v_reg = {'model': 'v ~ 1 + C(condition)', 'link_func': lambda x: x}

In [26]:
reg_descr = [z_reg, v_reg]

In [29]:
reg_descr

[{'model': 'z ~ 1 + C(condition)',
  'link_func': <function __main__.z_link_func(x, data=           rt  response  subj_idx condition  stimulus
0    1.018445       0.0         0    level1       1.0
1    1.556326       1.0         0    level1       1.0
2    0.903633       1.0         0    level1       1.0
3    0.658032       1.0         0    level1       1.0
4    0.866491       1.0         0    level1       1.0
..        ...       ...       ...       ...       ...
295  1.186306       1.0         9    level3       2.0
296  0.385079       0.0         9    level3       2.0
297  0.587854       0.0         9    level3       2.0
298  1.491150       0.0         9    level3       2.0
299  0.533947       0.0         9    level3       2.0

[300 rows x 5 columns])>},
 {'model': 'v ~ 1 + C(condition)',
  'link_func': <function __main__.<lambda>(x)>}]

In [30]:
mydata

,rt,response,subj_idx,condition,stimulus
0,1.018445,0.0,0,level1,1.0
1,1.556326,1.0,0,level1,1.0
2,0.903633,1.0,0,level1,1.0
3,0.658032,1.0,0,level1,1.0
4,0.866491,1.0,0,level1,1.0
...,...,...,...,...,...
295,1.186306,1.0,9,level3,2.0
296,0.385079,0.0,9,level3,2.0
297,0.587854,0.0,9,level3,2.0
298,1.491150,0.0,9,level3,2.0
